In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import regex as re
from contractions import fix
from sentence_transformers import SentenceTransformer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
true_df = pd.read_csv("test_data/True.csv")
print(true_df.shape)
true_df.info()
true_df.head()

(21417, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [3]:
true_df.drop(columns=["title","subject","date"],inplace=True)

In [4]:
true_df['text'] = true_df['text'].apply(lambda x: ' '.join(x.split(' - ')[1:]))

In [5]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", '', text)
    text = re.sub(r"\s+", ' ', text).strip()
    return text

In [6]:
true_df['text']=true_df['text'].apply(clean_text)
true_df.head()

,text
0,the head of a conservative republican faction ...
1,transgender people will be allowed for the fir...
2,the special counsel investigation of links bet...
3,trump campaign adviser george papadopoulos tol...
4,president donald trump called on the us postal...


In [7]:
small_true_df = true_df.sample(frac=0.1)
print(small_true_df.shape)

(2142, 1)


In [8]:
small_true_df["text"] = small_true_df["text"].apply(fix)

In [9]:
stop_words = set(stopwords.words('english'))
def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    return " ".join([word for word in tokens if word not in stop_words])

small_true_df['text'] = small_true_df['text'].apply(tokenize_and_remove_stopwords)

In [12]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [13]:
embeddings = model.encode(small_true_df['text'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/67 [00:00<?, ?it/s]

In [14]:
print(type(embeddings))
print(np.array(embeddings).shape)

<class 'numpy.ndarray'>
(2142, 384)


In [15]:
from sklearn.neighbors import LocalOutlierFactor

In [16]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01, novelty=True)
lof.fit(embeddings)

LocalOutlierFactor(contamination=0.01, novelty=True)

In [33]:
false_df = pd.read_csv("test_data/Fake.csv")
print(false_df.shape)
false_df.info()
false_df.head()

(23481, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [34]:
false_df.drop(columns=["title","subject","date"],inplace=True)
false_df['text'] = false_df['text'].apply(lambda x: ' '.join(x.split(' - ')[1:]))
false_df['text']=false_df['text'].apply(clean_text)

In [37]:
small_fake_df = false_df.sample(frac=0.005)
small_fake_df.shape

(117, 1)

In [38]:
small_fake_df["text"] = small_fake_df["text"].apply(fix)

In [39]:
small_fake_df['text'] = small_fake_df['text'].apply(tokenize_and_remove_stopwords)

In [40]:
embeddings_false = model.encode(small_fake_df['text'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [41]:
print(type(embeddings_false))
print(np.array(embeddings_false).shape)

<class 'numpy.ndarray'>
(117, 384)


In [42]:
predictions_test = lof.predict(embeddings_false)

In [44]:
num_anomalies = np.sum(predictions_test == -1)
num_normals = np.sum(predictions_test == 1)

print(f"Detected {num_anomalies} anomalies (fake news) out of {len(predictions_test)} samples")
print(f"Detected {num_normals} normal samples")

Detected 117 anomalies (fake news) out of 117 samples
Detected 0 normal samples


In [45]:
predictions_train = lof.predict(embeddings)

In [46]:
num_anomalies_train = np.sum(predictions_train == -1)
num_normals_train = np.sum(predictions_train == 1)

print(f"Detected {num_anomalies_train} anomalies (fake news) out of {len(predictions_train)} samples")
print(f"Detected {num_normals_train} normal samples")

Detected 15 anomalies (fake news) out of 2142 samples
Detected 2127 normal samples


In [47]:
import joblib
joblib.dump(lof, 'concept_draft_model.joblib')

['concept_draft_model.joblib']

In [49]:
from sklearn.ensemble import IsolationForest
model = IsolationForest(contamination=0.01, random_state=42)
model.fit(embeddings)

IsolationForest(contamination=0.01, random_state=42)

In [50]:
predictions_iso = model.predict(embeddings_false)

In [51]:
scores = model.decision_function(embeddings_false)

In [52]:
scores

array([0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395592,
       0.05395592, 0.05395592, 0.05395592, 0.05395592, 0.05395

In [53]:
from sklearn.neighbors import NearestNeighbors

In [55]:
nn = NearestNeighbors(n_neighbors=20)
nn.fit(embeddings)
distances, indices = nn.kneighbors([embeddings[0]])

neighbors_texts = small_true_df.iloc[indices[0]]['text'].tolist()

In [56]:
neighbors_distances = distances[0]

In [57]:
neighbors_texts

['us defense secretary jim mattis said thursday north korea missile launch japan put millions japanese duck cover landed pacific ocean added top us officials fully coordinated testlaunch got done calls always make coordinate among steady goes mattis told reporters traveling visit us strategic command oversees us nuclear forces',
 'united nations security council meet 3 pm edt 1900 gmt friday latest north korea missile test diplomats said request united states japan north korea fired missile flew japan northern hokkaido far pacific ocean friday south korean japanese officials said ratcheting tensions pyongyang recent test powerful nuclear bomb 15member security council unanimously stepped sanctions north korea monday sept 3 nuclear test imposing ban country textile exports capping imports crude oil ninth un sanctions resolution adopted north korea since 2006',
 'us president donald trump said monday japan would shoot north korean missiles sky bought us weaponry needed suggesting tokyo t

In [58]:
neighbors_distances

array([5.96046448e-08, 2.81875128e+00, 2.90641833e+00, 2.91303749e+00,
       3.02811230e+00, 3.10913668e+00, 3.12306978e+00, 3.13493599e+00,
       3.14435170e+00, 3.15806090e+00, 3.16265868e+00, 3.19118407e+00,
       3.20310609e+00, 3.20439443e+00, 3.21080607e+00, 3.23920879e+00,
       3.25043470e+00, 3.25443259e+00, 3.27424921e+00, 3.27555018e+00])